In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Familiarizing with the tools and data

Let's first analyze the HTML content of the IS-Academia directory. We will this by using the `requests` library to `GET` the HTML content given a URL. We obtained the following URL using Postman and Postman Interceptor.

In [5]:
r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247")

Now we will use `BeautifulSoup` to parse through the data and visualize it nicely with the `pretiffy()` method.

In [7]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <div>
   </div>
   <title>
   </title>
   <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
   </script>
   <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
   </link>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1">
     <input name="ww_i_reportmodel" type="hidden" value="133685247">
      <input name="ww_c_langue" type="hidden" value="">
       <h1 id="titre">
        Liste des étudiants inscrits par semestre
       </h1>
       <table border="0" id="format">
 

With `BeautifulSoup`, we can convinient dig deeper into the HTML content as described in this tutorial: https://www.crummy.com/software/BeautifulSoup/bs4/doc/. We will isolate the filters used to distinguish students by Major (Unité académique), Academic Year (Période académique), Student Status (Période pédagogique), and Semester Type (Type de semestre). We have identified from the output above that the filters are in the `body`, between `<table>` tags with `id="filtre"`. Finally, we can use `find_all('tr')` to get each filter as an entry in a list.

In [21]:
filters = soup.body.find(id="filtre").find_all('tr')
print(filters)

[<tr><th>Unit\xe9 acad\xe9mique</th><td><input name="zz_x_UNITE_ACAD" type="hidden" value=""><select name="ww_x_UNITE_ACAD" onchange="document.f.zz_x_UNITE_ACAD.value=document.f.ww_x_UNITE_ACAD.options[document.f.ww_x_UNITE_ACAD.selectedIndex].text"><option value="null"></option><option value="942293">Architecture</option><option value="246696">Chimie et g\xe9nie chimique</option><option value="943282">Cours de math\xe9matiques sp\xe9ciales</option><option value="637841336">EME (EPFL Middle East)</option><option value="942623">G\xe9nie civil</option><option value="944263">G\xe9nie m\xe9canique</option><option value="943936">G\xe9nie \xe9lectrique et \xe9lectronique </option><option value="2054839157">Humanit\xe9s digitales</option><option value="249847">Informatique</option><option value="120623110">Ing\xe9nierie financi\xe8re</option><option value="946882">Management de la technologie</option><option value="944590">Math\xe9matiques</option><option value="945244">Microtechnique</option

Admittedly, not the nicest output. And besides the fact that there are square brackets, it's hard to even say that it's a list! Let's try to output this more cleanly. From the `prettify()` output, we saw that each filter, e.g. `Architecture` for Unité académique or `2010-2011` for Période académique has an `option` tag surrounding it. Let's use `find_all(option)` with each item in the above list to cleanly output the filter options.

In [24]:
for field in filters:
    print(field.find_all('option'))

[<option value="null"></option>, <option value="942293">Architecture</option>, <option value="246696">Chimie et g\xe9nie chimique</option>, <option value="943282">Cours de math\xe9matiques sp\xe9ciales</option>, <option value="637841336">EME (EPFL Middle East)</option>, <option value="942623">G\xe9nie civil</option>, <option value="944263">G\xe9nie m\xe9canique</option>, <option value="943936">G\xe9nie \xe9lectrique et \xe9lectronique </option>, <option value="2054839157">Humanit\xe9s digitales</option>, <option value="249847">Informatique</option>, <option value="120623110">Ing\xe9nierie financi\xe8re</option>, <option value="946882">Management de la technologie</option>, <option value="944590">Math\xe9matiques</option>, <option value="945244">Microtechnique</option>, <option value="945571">Physique</option>, <option value="944917">Science et g\xe9nie des mat\xe9riaux</option>, <option value="942953">Sciences et ing\xe9nierie de l'environnement</option>, <option value="945901">Science

That's a bit better. We can see a `value` associated with each field option, e.g. `249847` for `Informatique`. Let's actually use some of these filters on the website itself and intercept the requests using Postman + Postman Interceptor. Postman helps us track which URL's are requested and to analyze the corresponding HTML. Let's check out the URL and HTML content (with `BeautifulSoup`) when we select the following field options: `Informatique`, `2009-2010`, `Bachelor semestre 1`, and `Semestre d'automne`.

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=2009-2010&ww_x_PERIODE_ACAD=978195&zz_x_PERIODE_PEDAGO=Bachelor+semestre+1&ww_x_PERIODE_PEDAGO=249108&zz_x_HIVERETE=Semestre+d%27automne&ww_x_HIVERETE=2936286&dummy=ok

In the URL we can see the options we selected! Moreover, they have been used as parameters for the URL along with their corresponding `value` attribute. The parameter names (that `Informatique`, `2009-2010`, `Bachelor semestre 1`, and `Semestre d'automne` are being set to) could be identified by navigating through our `filters` list (by going into the `td` tag and then selecting the `name` attribute of the `input` tag).

In [31]:
for field in filters:
    print(field.td.input["name"])

zz_x_UNITE_ACAD
zz_x_PERIODE_ACAD
zz_x_PERIODE_PEDAGO
zz_x_HIVERETE


`zz_*` seems to be for the string parameter name and `xx_*` for the corresponding `value` attribute. However, it is also possible to get the same HTML content without the `zz_*` parameters by adding the `ww_b_list` parameter (we found this out using Postmaster):

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_i_reportModelXsl=133685270&ww_x_HIVERETE=2936286&ww_x_PERIODE_ACAD=978195&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_PEDAGO=249108

When checking out the HTML content, we see a new table at the bottom (corresponding to the options we see on the IS-Academia portal) with attribute `border="0"`. Let's check it out with `BeautifulSoup`.

In [34]:
r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=2009-2010&ww_x_PERIODE_ACAD=978195&zz_x_PERIODE_PEDAGO=Bachelor+semestre+1&ww_x_PERIODE_PEDAGO=249108&zz_x_HIVERETE=Semestre+d%27automne&ww_x_HIVERETE=2936286&dummy=ok")
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <div>
   </div>
   <title>
   </title>
   <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
   </script>
   <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
   </link>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1">
     <input name="ww_i_reportmodel" type="hidden" value="133685247">
      <input name="ww_c_langue" type="hidden" value="">
       <h1 id="titre">
        Liste des étudiants inscrits par semestre
       </h1>
       <table border="0" id="format">
 

We now see this new parameter `ww_x_GPS`. Let's follow the link for `Informatique, 2009-2010, Bachelor semestre 1` and analyze as before. The webpage now shows the corresponding list of students! With Postman, we see a `GET` request with the following URL:

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=213617925&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=978195&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=2936286

This is very similar to previous URL with one key difference: the new parameter `ww_x_GPS` with its corresponding value has been added to the URL.

We have now "cracked" the manner in which to extract the desired HTML content from IS-Academia! The general procedure is as follows:

1. Identity the `value` attributes according to desired filters.
2. Using `requests`, build the URL for filter search results with the `value` attributes as parameters of the following base URL: http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270
3. Use `BeautifulSoup` to extract the `ww_x_GPS` parameter value from the HTML content.
4. With `requests`, build the URL with the newly acquired `ww_x_GPS` value and the `value` attributes as parameters of the following base URL: http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1
5. We then have a table of students in HTML format. We can use the `read_html()` function of `pandas` in order to conveniently access the data.

Below we will go through the above steps for picking out the students we need for our analysis in the exercises.

#### 1. Identify `value` attributes according to desired filters

In order to perform the first step conveniently, we will create a few dictionaries so we can "translate" the desired filter options into their corresponding `value` attributes. These dictionaries will be used in the following exercises.

In [40]:
# same URL as before, identified with postmam
r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247")
# scrape content using BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')
# obtain list of filters as previously described
filters = soup.body.find(id="filtre").find_all('tr')

Now we define a new function `create_dict()` in order to scrape the string and corresponding `value` attribute from a list of `option`'s. The function will finally place them in a dictionary with the string as the key and the `value` attribute as the (you, got it) value, and then return this newly formed dictionary.

In [43]:
# function to create dictionary for each filter
def create_dict(field_list):
    field_dict = {}
    for i in range(1, len(field_list)):
        field_dict[field_list[i].string] = field_list[i]["value"]
    return field_dict

# Unité académique, Période académique, Période pédagogique, Type de semestre
major_dict = create_dict(filters[0].find_all('option'))
acad_yr_dict = create_dict(filters[1].find_all('option'))
status_dict = create_dict(filters[2].find_all('option'))
sem_dict = create_dict(filters[3].find_all('option'))

Let's create a Series from each of these dictionaries and `pickle` them so we don't have to have to rely on `requests`.

In [48]:
major = pd.Series(data=major_dict)
major.to_pickle("major_pickle")
acad_yr = pd.Series(data=acad_yr_dict)
acad_yr.to_pickle("acad_yr_pickle")
status = pd.Series(data=status_dict)
status.to_pickle("status_pickle")
sem = pd.Series(data=sem_dict)
sem.to_pickle("sem_pickle")

Now we can convieniently obtain the necessary parameters to build the URLs for filtering students based on Major (Unité académique), Academic Year (Période académique), Student Status (Période pédagogique), and Semester Type (Type de semestre)!

#### 2. Using `requests`, build the URL for filter search results with the `value` attributes as parameters

Now let's build the required URL so we can obtain the `ww_x_GPS` parameter value to then gather the students that meet our search criteria. The following is our base URL for the filter search results:

In [118]:
FILTER_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270'

Now using `requests`, we can build the URL with the necessary parameters as we saw above. We have the following parameters:

In [50]:
# parameter keys
PARAM_MAJOR = 'ww_x_UNITE_ACAD'
PARAM_YR = 'ww_x_PERIODE_ACAD'
PARAM_STATUS = 'ww_x_PERIODE_PEDAGO'
PARAM_SEM = 'ww_x_HIVERETE'

Now let's pass parameters to the URL as described here (http://docs.python-requests.org/en/master/user/quickstart/#passing-parameters-in-urls) and make a `GET` request. Let's say we want students in `Informatique`, `2009-2010`, `Bachelor semestre 1`, and `Semestre d'automne` as before.

In [119]:
# create URL for filtered result
payload_filter = {PARAM_MAJOR: major['Informatique'], 
                  PARAM_YR: acad_yr['2009-2010'], 
                  PARAM_STATUS: status['Bachelor semestre 1'], 
                  PARAM_SEM: sem["Semestre d'automne"]}
r = requests.get(FILTER_BASE_URL, params=payload_filter)
print(r.url)

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&ww_x_PERIODE_PEDAGO=249108&ww_x_PERIODE_ACAD=978195&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=2936286


#### 3. Use `BeautifulSoup` to extract the `ww_x_GPS` parameter value from the HTML content. 

Let's see how we can use `BeautifulSoup` to navigate through the HTML content and extract the `ww_x_GPS` parameter value. `prettify()` can help us with this.

In [120]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <div>
   </div>
   <title>
   </title>
   <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
   </script>
   <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
   </link>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1">
     <input name="ww_i_reportmodel" type="hidden" value="133685247">
      <input name="ww_c_langue" type="hidden" value="">
       <h1 id="titre">
        Liste des étudiants inscrits par semestre
       </h1>
       <table border="0" id="format">
 

We need to pick out the `a` tags that have a `class` attribute equal to `ww_x_GPS`. This can be done with the `find_all()` method.

In [121]:
soup.find_all('a', class_='ww_x_GPS')

[<a class="ww_x_GPS" href="javascript:void(0)" onclick="loadReport('ww_x_GPS=-1');return false;">Tous</a>,
 <a class="ww_x_GPS" href="javascript:void(0)" onclick="loadReport('ww_x_GPS=213617925');return false;">Informatique, 2009-2010, Bachelor semestre 1</a>]

Now we have a list of HTML entries that contain `ww_x_GPS` values. The value itself is in the `onclick` attribute. We can extract the `ww_x_GPS` value by parsing the information contained in this attribute. We will assume that we only get two entries in the above list are our filter entries as our search criteria will ensure this. The two categories (which can be seen on the IS-Academia site) are "Tous" and the category of students we are interested in. "Tous" has a `ww_x_GPS` value of `-1` so we make sure to return the other value.

In [122]:
# assuming we only get two results with one of them being "Tous"
def is_valid_gps(gps):
    return gps != "-1"

def extract_gps(content):
    soup = BeautifulSoup(content, 'html.parser')
    elements = soup.find_all('a', class_='ww_x_GPS')
    for element in elements:
        raw_info = element.attrs['onclick']
        gps = raw_info.split("'")[1].split('=')[1]
        if is_valid_gps(gps):
            return gps

gps = extract_gps(r.content)
print(gps)

213617925


#### 4. With `requests`, build the URL with the newly acquired `ww_x_GPS` value and the `value` attributes as parameters. 

Now we have a new base URL and an additional parameter for our payload.

In [123]:
PARAM_GPS = 'ww_x_GPS'
DATA_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1'

As in Step 2, we use `requests` to can build the URL with the necessary parameters.

In [124]:
# create URL for filtered result
payload_data = {PARAM_GPS: gps,
                PARAM_MAJOR: major['Informatique'], 
                PARAM_YR: acad_yr['2009-2010'], 
                PARAM_STATUS: status['Bachelor semestre 1'], 
                PARAM_SEM: sem["Semestre d'automne"]}
r = requests.get(DATA_BASE_URL, params=payload_data)
print(r.url)

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1&ww_x_PERIODE_PEDAGO=249108&ww_x_PERIODE_ACAD=978195&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=2936286&ww_x_GPS=213617925


Following the above link takes us to the list of students meeting the following criteria: `Informatique`, `2009-2010`, `Bachelor semestre 1`, and `Semestre d'automne`.

#### 5. Use the `read_html()` function of `pandas` (or `BeautifulSoup`) in order to conveniently access the data.

##### Using with `pandas`

`read_html()` stores HTML tables as a list of `DataFrame`'s. So we need to access the first element for our desired table.

In [125]:
dfs = pd.read_html(r.url)
dfs[0].head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Informatique, 2009-2010, Bachelor semestre 1 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN
2,Monsieur,Abdallah Jad,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,194197,NaN
3,Madame,Al Azawi Marwa,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,195766,NaN
4,Monsieur,Amrani Ismaïl,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,186942,NaN
5,Monsieur,Antognini Marco,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,194182,NaN
6,Monsieur,Augsburger Damien,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,186595,NaN
7,Madame,Balmau Oana Maria,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,192757,NaN
8,Monsieur,Barben Loïc,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,189517,NaN
9,Monsieur,Barbier Issa,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,192248,NaN


The first two rows corresponds to information about the students. After the first two rows, we have the student name, gender, SCIPER, etc. Therefore, the some pre-processing needs to be done. These are some example pre-processing steps (perhaps more or less is needed depending on what info we want to extract): remove first row, set second row as the column names, and set SCIPER number of index (since this is a unique ID).

In [126]:
df = dfs[0].copy()
df.columns = df.iloc[1] # set columns names
df = df.reindex(df.index.drop([0,1])) # drop non-student rows
df = df.set_index('No Sciper')
df.head(10)

1,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,nan
No Sciper,,,,,,,,,,,
194197,Monsieur,Abdallah Jad,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
195766,Madame,Al Azawi Marwa,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
186942,Monsieur,Amrani Ismaïl,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
194182,Monsieur,Antognini Marco,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
186595,Monsieur,Augsburger Damien,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
192757,Madame,Balmau Oana Maria,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
189517,Monsieur,Barben Loïc,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
192248,Monsieur,Barbier Issa,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN
187143,Monsieur,Bennani Kabchi Reda,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN


##### Using `BeautifulSoup`

One problem with using the `read_html()` function is that it (again) fetches the HTML content although this was already one with `resquests`. (We could alternatively build the full URL without `requests.get()` and just fetch the HTML content once). We can also scrape the HTM content using `BeautifulSoup`.

In [128]:
soup_students = BeautifulSoup(r_list.content, 'html.parser')
# visualize
print(soup_students.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <link href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css"/>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <fieldset style="text-align:right; width:40%; position:relative; margin-right: 10px;float:right; border: 0; padding: 0 0 8px 0;">
   <a href="!GEDREPORTS.html?ww_i_reportModel=133685247&amp;ww_i_reportModelXsl=133685270&amp;ww_b_list=1&amp;ww_x_PERIODE_PEDAGO=249108&amp;ww_x_PERIODE_ACAD=355925344&amp;ww_x_UNITE_ACAD=249847&amp;ww_x_HIVERETE=2936286&amp;ww_x_GPS=2021043255" style="color:#990033;">
    Identification pour accéder aux e-mails
    <br>
     Login to access email adresses
    </br>
   </a>
  </fieldset>
  <script>
   function mailList(x) {
   var vtop = (screen.height-200)/2;
   var vleft=(screen.width-600)/2;
   var w=open("", "emaillist", "Scrollbars=1,resizabl

From the `prettify()` output, we see that student info is contained within `<tr>` tags and that the first two entries between `<tr>` are for general information about the students. Therefore, to get all the students, we can use `find_all()` to get all the `<tr>` entries and drop the first two.

In [132]:
students = soup_students.find_all('tr')[2:]
# let's look at one of the student entries
students[0]

<tr><td style="white-space:nowrap">Monsieur</td><td style="white-space:nowrap">Abbey\xa0Alexandre</td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap">Pr\xe9sent</td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td>235688</td><td style="white-space:nowrap"></td></tr>

Each data point about the student is surround by `<td>` tags. We can again use `find_all()` to access these elements and print these nicely.

In [135]:
student = students[0].find_all('td')
for field in student:
    print(field.string)

Monsieur
Abbey Alexandre
None
None
None
None
None
Présent
None
None
235688
None


# Exercise 1

_We will focus exclusively on the academic unit `Informatique`._

_Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both `Bachelor semestre 1` and `Bachelor semestre 6`. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?_

For this problem, we will create two dictionaries: one for `Bachelor semestre 1` (B1) students and another for `Bachelor semestre 6` (B6) students. For the B1 dictionary, we will start from 2007 and add students when they are first enrolled as `Informatique` and B1. If they repeat, they will not be added again. The same will be done for B6 as the problem is to compute the number of months from first to sixth semester. To compute the months, we will take into account the academic year: **12*[(second of the academic years for B6) - (first of the academic years for B1)]**.

A few assumptions we make:
* `Bachelor semestre 1` is always in `Semestre d'automne` due to the way the Bachelor program is structured.
* `Bachelor semestre 6` is always in `Semestre de printemps` due to the way the Bachelor program is structured.

**NOTE** : We realize that there is some discussion on Slack with people considering students that finished their studies with `Bachelor semestre 5`; however, the assignment states to compute the number of months for each student to go from the first to the sixth semester (how long it took for their Bachelors is a different problem).

In [158]:
## (REMINDER) constants and functions as previously described/explained
# parameter keys
PARAM_GPS = 'ww_x_GPS'
PARAM_MAJOR = 'ww_x_UNITE_ACAD'
PARAM_YR = 'ww_x_PERIODE_ACAD'
PARAM_STATUS = 'ww_x_PERIODE_PEDAGO'
PARAM_SEM = 'ww_x_HIVERETE'

# base urls
FILTER_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270'
DATA_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1'

# extracting GPS value
def is_valid_gps(gps):
    return gps != "-1"
def extract_gps(content):
    soup = BeautifulSoup(content, 'html.parser')
    elements = soup.find_all('a', class_='ww_x_GPS')
    for element in elements:
        raw_info = element.attrs['onclick']
        gps = raw_info.split("'")[1].split('=')[1]
        if is_valid_gps(gps):
            return gps

# combine steps 1-4 from procedure of extracting desired students
def get_html_content(major, year, status, sem):
    # obtain gps
    payload = {PARAM_MAJOR: major_dict[major],
                      PARAM_YR: acad_yr_dict[year], 
                      PARAM_STATUS: status_dict[status],
                      PARAM_SEM: sem_dict[sem]}
    r_filt = requests.get(FILTER_BASE_URL, params=payload)
    gps = extract_gps(r_filt.content)
    # get list of students
    payload[PARAM_GPS] = gps
    r_list = requests.get(DATA_BASE_URL, params=payload)
    return r_list
        
# read HTML content and pre-process using steps described above
def create_student_list(url):
    df = pd.read_html(url)[0]
    df.columns = df.iloc[1] # set columns names
    df = df.reindex(df.index.drop([0,1])) # drop non-student rows
    df = df.set_index('No Sciper')
    return df

In [ ]:
# open the Series for the filter dropdown menus made before
major = pd.read_pickle("major_pickle")
acad_yr = pd.read_pickle("acad_yr_pickle")
status = pd.read_pickle("status_pickle")
sem = pd.read_pickle("sem_pickle")

Let's first create the dictionary for the `Bachelor semestre 6` students.

In [169]:
b6_students = {}
# follow steps outlined in previous section for obtaining desired HTML table
for year in acad_yr.index:
    end_year = int(year.split('-')[1])
    # obtain html url with student list
    r_list = get_html_content('Informatique', year, 'Bachelor semestre 6', 'Semestre de printemps')
    # IN THIS CASE EASIER TO USE BEAUTIFUL SOUP APPROACH
    soup_students = BeautifulSoup(r_list.content, 'html.parser')
    # students are starting after two rows
    students = soup_students.find_all('tr')
    for k in range(2,len(students)):
        test_student = students[k].find_all('td')
        sciper = test_student[10].string
        gender = test_student[0].string
        status = test_student[7].string
        # replace with latest year in case 6th semester was repeated
        b6_students[sciper] = [gender, status, end_year]

Let's use this dictionary to make a `DataFrame` which can be conviently used for manipulation and stored as a `pickle` for future use.

In [171]:
df_b6 = pd.DataFrame.from_dict(b6_students, orient='index')
df_b6.columns = ['Gender', 'Status', 'End_Year']
df_b6.index.rename('SCIPER', inplace=True)
df_b6.to_pickle("df_b6_pickle")
df_b6.head()

,Gender,Status,End_Year
SCIPER,,,
204222,Monsieur,Présent,2015
186264,Monsieur,Présent,2013
161279,Monsieur,Présent,2009
227848,Monsieur,Présent,2014
216778,Madame,Congé,2014


Now let's go through the `Bachelor semestre 1` students.

In [172]:
b1_students = {}
# follow steps outlined in previous section for obtaining desired HTML table
for year in acad_yr.index:
    start_year = int(year.split('-')[0])
    # obtain html url with student list
    r_list = get_html_content('Informatique', year, 'Bachelor semestre 1', "Semestre d'automne")
    # parse with beautiful soup
    soup_students = BeautifulSoup(r_list.content, 'html.parser')
    # students are starting after two rows - IN THIS CASE EASIER TO USE BEAUTIFUL SOUP APPROACH
    students = soup_students.find_all('tr')
    for k in range(2,len(students)):
        test_student = students[k].find_all('td')
        sciper = test_student[10].string
        # keep earliest year in case a student repeated first semester of Bachelor year
        if sciper not in b1_students:
            # these fields were determined before
            gender = test_student[0].string
            status = test_student[7].string
            b1_students[sciper] = [gender, status, start_year]

Let's use this dictionary to make a `DataFrame` which can be conviently used for manipulation and stored as a `pickle` for future use.

In [188]:
df_b1 = pd.DataFrame.from_dict(b1_students, orient='index')
df_b1.columns = ['Gender', 'Status', 'Start_Year']
df_b1.index.rename('SCIPER', inplace=True)
df_b1.to_pickle("df_b1_pickle")
df_b1.head()

,Gender,Status,Start_Year
SCIPER,,,
198197,Monsieur,Présent,2010
204222,Monsieur,Présent,2011
186264,Monsieur,Présent,2008
261729,Monsieur,Présent,2015
239596,Monsieur,Présent,2013


In [199]:
# join b1 and b6 students and keep those that are in both, also drop Status column
grads = df_b1.join(df_b6.End_Year, how='inner').drop('Status', axis=1)
grads.head()

,Gender,Start_Year,End_Year
SCIPER,,,
204222,Monsieur,2011,2015
186264,Monsieur,2008,2013
216778,Madame,2011,2014
223566,Monsieur,2012,2015
186260,Monsieur,2009,2014


In [201]:
# compute duration
def compute_duration(x):
    return x['End_Year']-x['Start_Year']

grads['Duration'] = grads.apply(compute_duration, axis=1)
grads.head()

,Gender,Start_Year,End_Year,Duration
SCIPER,,,,
204222,Monsieur,2011,2015,4
186264,Monsieur,2008,2013,5
216778,Madame,2011,2014,3
223566,Monsieur,2012,2015,3
186260,Monsieur,2009,2014,5


In [219]:
male_mean = grads.Duration[grads.Gender=="Monsieur"].mean()
female_mean = grads.Duration[grads.Gender=="Madame"].mean()
print("Average duration for male students: " + str(male_mean))
print("Average duration for female students: " + str(female_mean))
print(grads.Gender.value_counts())

Average duration for male students: 3.48097826087
Average duration for female students: 3.31034482759
Monsieur    368
Madame       29
Name: Gender, dtype: int64


In [218]:
print(df_b1.Gender.value_counts())
print(df_b6.Gender.value_counts())

Monsieur    1197
Madame       126
Name: Gender, dtype: int64
Monsieur    481
Madame       35
Name: Gender, dtype: int64


In [221]:
# Two-Sample T-Test
import scipy.stats as stats
stats.ttest_ind(a= grads.Duration[grads.Gender=="Monsieur"],
                b= grads.Duration[grads.Gender=="Madame"],
                equal_var=False) 

Ttest_indResult(statistic=1.5831651359439454, pvalue=0.12191236829650295)

The test yields a p-value of 0.1219, which means there is a 12.19% chance we'd see sample data this far apart if the two groups tested are actually identical. If we were using a 95% confidence level we would fail to reject the null hypothesis (being that the means are the same), since the p-value is greater than the corresponding significance level of 5%. Therefore, the difference in average duration is not statistically significant.